# Time Chunking Utility Development Python Notebook

In [13]:
import pandas as pd
import numpy as np
from preprocess import * 


In [184]:
csop = pd.read_csv('../data/raw_data/csopII_conversations_withblanks.csv')
csop = preprocess_conversation_columns(csop)
csop.head()

,conversation_num,batch_num,vis_img,int_verb,ort_img,rep_man,soc_pers,team_size,difficulty,score,duration,efficiency,speaker_nickname,message,timestamp
0,rHPaiuXkM3Ss4rEsW_easy,1,High,High,High,High,High,5,Easy [Corresponds to 'Hard' in PNAS],586.0,107.0,5.476636,oJLD2kfcwGyXkfCeH,Can we get D to the highest score?,2021-02-26T19:11:21.474Z
1,dArSAcrzmb9bR6Pug_easy,1,High,High,High,High,High,5,Easy [Corresponds to 'Hard' in PNAS],559.0,406.0,1.376847,6QvNn8wdCyviKCjiv,looks good,2021-02-26T19:22:32.695Z
2,dArSAcrzmb9bR6Pug_easy,1,High,High,High,High,High,5,Easy [Corresponds to 'Hard' in PNAS],559.0,406.0,1.376847,6qP7EcqvYQL4gmrAX,I like it,2021-02-26T19:23:14.467Z
3,dArSAcrzmb9bR6Pug_easy,1,High,High,High,High,High,5,Easy [Corresponds to 'Hard' in PNAS],559.0,406.0,1.376847,NMkPLqERKzTTMqw5Y,It looks good to me,2021-02-26T19:23:20.650Z
4,dArSAcrzmb9bR6Pug_easy,1,High,High,High,High,High,5,Easy [Corresponds to 'Hard' in PNAS],559.0,406.0,1.376847,6QvNn8wdCyviKCjiv,stop changing it,2021-02-26T19:23:52.111Z


In [183]:
negotiation = pd.read_csv('../data/raw_data/negotiation_test_data.csv')
negotiation = preprocess_conversation_columns(negotiation)
negotiation.head()

,conversation_num,message,timestamp,speaker_nickname
0,01H8HEHNZHXV8X6B8DPQE2TC5K,Hey guys!,10,01H8HEJT1YXZMNZP0M446TS7X7
1,01H8HEHNZHXV8X6B8DPQE2TC5K,Hi everyone!,11,01H8HEJ55AFY9VBJYY0KPNDTK9
2,01H8HEHNZHXV8X6B8DPQE2TC5K,Hey there,12,01H8HEJC8KA1P3PBXMDXT5K6FW
3,01H8HEHNZHXV8X6B8DPQE2TC5K,"So I represent organization A, and it's pretty...",20,01H8HEJT1YXZMNZP0M446TS7X7
4,01H8HEHNZHXV8X6B8DPQE2TC5K,Honestly A how about we just make a deal for t...,25,01H8HEJ55AFY9VBJYY0KPNDTK9


In [185]:
juries = pd.read_csv('../data/raw_data/jury_conversations_with_outcome_var.csv')
juries = preprocess_conversation_columns(juries)
juries.head()

,conversation_num,batch_num,round_num,speaker_nickname,timestamp,message,majority_pct,num_flipped,flipped_pct,num_votes
0,0,0,0,5e7e1e0031f4e454e196c30b,2020-04-20T18:27:20.125Z,Hello!,1.0,1,0.333333,3
1,0,0,0,5e31d6e4e31c5304c46f1413,2020-04-20T18:27:23.764Z,Hi!,1.0,1,0.333333,3
2,0,0,0,5e7e4f4c31f4e454e196c9c4,2020-04-20T18:27:27.724Z,Hello,1.0,1,0.333333,3
3,0,0,0,5d482ea421c9be351f762255,2020-04-20T18:27:30.410Z,Hi,1.0,1,0.333333,3
4,0,0,0,5e84cc3c50f6e364321d6265,2020-04-20T18:27:35.506Z,hi,1.0,1,0.333333,3


In [205]:
pgg = pd.read_csv('../data/raw_data/pgg_conversations_withblanks.csv')
pgg = preprocess_conversation_columns(pgg)
pgg.head()

,conversation_num,treatmentName,punishmentCost,punishmentMagnitude,total_costs,total_penalties,total_rewards,multiplier,punishmentExists,rewardExists,playerCount,numRounds,message,timestamp,speaker_nickname,score
0,MnarujjeewXn9m7sz,4_T,3.0,9.0,1419,27,4230,5.6,True,True,6,15,NaN,0,NULL_SPEAKER,1.258631
1,u5ha44udM2BMXBZnj,1_C,2.0,4.0,3280,0,6560,4.6,False,True,6,35,NaN,0,NULL_SPEAKER,1.143375
2,gSojrSFyc4ASdFmCt,8_C,2.0,7.0,0,0,0,1.0,False,False,3,10,NaN,0,NULL_SPEAKER,1.003333
3,vEWvuFNWzMJuyGdrH,0_T,3.0,9.0,0,0,0,1.3,True,False,2,20,"Nice, If we contribute all of our coins each t...",1677262097074,L5Sy83kBiyXoDhg6D,1.000000
4,vEWvuFNWzMJuyGdrH,0_T,3.0,9.0,0,0,0,1.3,True,False,2,20,Sounds good.,1677262112288,zn7zNZvgQjhN4Qsty,1.000000


In [200]:
def create_chunks(df,num_chunks):

    final_df = pd.DataFrame(columns=df.columns)

    # Replace instances of NULL_TIME; this throws off the type checking
    df['timestamp'] = df['timestamp'].replace('NULL_TIME', None)
    timestamps = df['timestamp'].dropna()

    is_datetime_string = False

    # Check the type of the timestamp string
    if (isinstance(timestamps[0], str)): # DateTime String, e.g., '2023-02-20 09:00:00'
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        is_datetime_string = True
    elif(isinstance(timestamps[0], int)):
        if(timestamps[0] > 423705600): # this is Unix time; the magic number is a time in 1983!
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        # If it's not Unix time, we can treat it as an int offset

    # Group and calculate difference
    for conversation_num, group in df.groupby(['conversation_num']):

        if pd.api.types.is_datetime64_any_dtype(df['timestamp']):
            total_duration_seconds = (group['timestamp'].max() - group['timestamp'].min()).total_seconds() if is_datetime_string else (group['timestamp'].max() - group['timestamp'].min()).total_seconds()
        else:
            total_duration_seconds = (group['timestamp'].max() - group['timestamp'].min())

        # Add a new column for chunk number
        group['chunk'] = -1 

        # Catch NA's from the case where people didn't chat or only 1 chat exists --- all chunk nums should be 0
        if pd.isna(total_duration_seconds) or total_duration_seconds == 0:
            group['chunk'] = 0

        # Case where people did chat
        else:
            #print(total_duration_seconds)
            #print(f"total duration be like {total_duration}")

            # Calculate the duration of each chunk
            chunk_duration = total_duration_seconds / num_chunks
            #print(f"chunk duration be like {chunk_duration}")

            # Initialize the 'chunk' column
            group['chunk'] = -1

            for index, row in group.iterrows():
                # Get the timestamp
                timestamp = row['timestamp']
                # Calculate the chunk number
                if pd.api.types.is_datetime64_any_dtype(df['timestamp']):
                    chunk_number = int(((timestamp - group['timestamp'].min()).total_seconds()) / chunk_duration)
                else:
                    chunk_number = int(((timestamp - group['timestamp'].min())) / chunk_duration)
                # print(f"chunk num be like {chunk_number}")

                # Assign the chunk number for each row
                group.loc[index, 'chunk'] = chunk_number

            # restrict the range of the chunks from 0 to num_chunks - 1
            group['chunk'] = group['chunk'].clip(0, num_chunks - 1)

        final_df = pd.concat([final_df, group], ignore_index=True)

    return final_df

In [206]:
# case where chunks are in unix time
create_chunks(pgg, 3)

,conversation_num,treatmentName,punishmentCost,punishmentMagnitude,total_costs,total_penalties,total_rewards,multiplier,punishmentExists,rewardExists,playerCount,numRounds,message,timestamp,speaker_nickname,score,chunk
0,AsksqjZ3o5evkoYXu,2_T,1.0,1.0,256,245,11,3.2,True,True,8,5,NaN,0,NULL_SPEAKER,0.482812,0.0
1,CgLiTN2jqAdKetEW4,3_T,4.0,13.0,4,13,0,4.0,True,False,4,45,NaN,0,NULL_SPEAKER,0.996736,0.0
2,EZEPLDywvah6husHW,2_C,1.0,1.0,25,0,25,3.2,False,True,8,5,NaN,0,NULL_SPEAKER,0.654687,0.0
3,EqiikgYjTFRyueZsm,5_T,1.0,2.0,1866,3732,0,2.4,True,False,10,40,hi everyone,1677262940272,5dQrXBdm4SgzEqKeE,0.554323,0.0
4,EqiikgYjTFRyueZsm,5_T,1.0,2.0,1866,3732,0,2.4,True,False,10,40,Hiya :),1677262944109,EmY6Asi7ngDtC9mPS,0.554323,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,ufttuN8GvJi76BD9G,5_C,1.0,2.0,0,0,0,2.4,False,False,10,40,anyone remember how many rounds?,1677263804045,wjwKAzezsvKEKzoZC,0.962604,2.0
366,vEWvuFNWzMJuyGdrH,0_T,3.0,9.0,0,0,0,1.3,True,False,2,20,"Nice, If we contribute all of our coins each t...",1677262097074,L5Sy83kBiyXoDhg6D,1.000000,0.0
367,vEWvuFNWzMJuyGdrH,0_T,3.0,9.0,0,0,0,1.3,True,False,2,20,Sounds good.,1677262112288,zn7zNZvgQjhN4Qsty,1.000000,2.0
368,xi8b3rnqAESQAEGS8,7_T,3.0,12.0,60,240,0,1.8,True,True,4,30,NaN,0,NULL_SPEAKER,0.758333,0.0


In [201]:
# Case where chunks are just regular ints
create_chunks(negotiation, 3)

,conversation_num,message,timestamp,speaker_nickname,chunk
0,01H8HEHNZHXV8X6B8DPQE2TC5K,Hey guys!,10,01H8HEJT1YXZMNZP0M446TS7X7,0.0
1,01H8HEHNZHXV8X6B8DPQE2TC5K,Hi everyone!,11,01H8HEJ55AFY9VBJYY0KPNDTK9,0.0
2,01H8HEHNZHXV8X6B8DPQE2TC5K,Hey there,12,01H8HEJC8KA1P3PBXMDXT5K6FW,0.0
3,01H8HEHNZHXV8X6B8DPQE2TC5K,"So I represent organization A, and it's pretty...",20,01H8HEJT1YXZMNZP0M446TS7X7,1.0
4,01H8HEHNZHXV8X6B8DPQE2TC5K,Honestly A how about we just make a deal for t...,25,01H8HEJ55AFY9VBJYY0KPNDTK9,1.0
5,01H8HEHNZHXV8X6B8DPQE2TC5K,Wait hold on,30,01H8HEJC8KA1P3PBXMDXT5K6FW,2.0
6,01H8HEHNZHXV8X6B8DPQE2TC5K,You can't just leave me out in the cold like that,31,01H8HEJC8KA1P3PBXMDXT5K6FW,2.0
7,01H8HEHNZHXV8X6B8DPQE2TC5K,Why not? You only contribute like 3 points lol,33,01H8HEJT1YXZMNZP0M446TS7X7,2.0
8,01H8HEHNZHXV8X6B8DPQE2TC5K,Wait A we're splitting the 118 evenly right,34,01H8HEJ55AFY9VBJYY0KPNDTK9,2.0
9,01H8HEHNZHXV8X6B8DPQE2TC5K,What? no way,37,01H8HEJT1YXZMNZP0M446TS7X7,2.0


In [202]:
# Case where chunks are datetime strings
create_chunks(csop, 3)

,conversation_num,batch_num,vis_img,int_verb,ort_img,rep_man,soc_pers,team_size,difficulty,score,duration,efficiency,speaker_nickname,message,timestamp,chunk
0,25PJSrbtr8mkHTvcw_easy,1,Mixed,High,Low,High,Mixed,2,Easy [Corresponds to 'Hard' in PNAS],604.0,559.0,1.080501,NiBqtamR7LYPByEoz,I think the next combination above 500 that wo...,2021-03-19 18:20:28.277000+00:00,0.0
1,25PJSrbtr8mkHTvcw_easy,1,Mixed,High,Low,High,Mixed,2,Easy [Corresponds to 'Hard' in PNAS],604.0,559.0,1.080501,NiBqtamR7LYPByEoz,I think 504 would still be ok?,2021-03-19 18:22:33.265000+00:00,2.0
2,25PJSrbtr8mkHTvcw_easy,1,Mixed,High,Low,High,Mixed,2,Easy [Corresponds to 'Hard' in PNAS],604.0,559.0,1.080501,NiBqtamR7LYPByEoz,604-100,2021-03-19 18:22:41.278000+00:00,2.0
3,25PJSrbtr8mkHTvcw_easy,1,Mixed,High,Low,High,Mixed,2,Easy [Corresponds to 'Hard' in PNAS],604.0,559.0,1.080501,hF7iWkQvsomPnXY7S,sure,2021-03-19 18:22:50.937000+00:00,2.0
4,25PJSrbtr8mkHTvcw_hard,1,Mixed,High,Low,High,Mixed,2,Hard [Corresponds to 'Super Hard' in PNAS],729.0,179.0,4.072626,NULL_SPEAKER,NaN,NaT,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4614,zwtfREfmLE7LzyW7n_easy,1,High,Mixed,Mixed,Mixed,High,4,Easy [Corresponds to 'Hard' in PNAS],537.0,294.0,1.826531,ttp99Mg4k27H4G4g2,you have ruined it all,2021-03-11 18:33:02.259000+00:00,2.0
4615,zwtfREfmLE7LzyW7n_easy,1,High,Mixed,Mixed,Mixed,High,4,Easy [Corresponds to 'Hard' in PNAS],537.0,294.0,1.826531,eiZmgSepzvcWzXZnq,IT WAS FINE BEFORE,2021-03-11 18:33:11.705000+00:00,2.0
4616,zwtfREfmLE7LzyW7n_easy,1,High,Mixed,Mixed,Mixed,High,4,Easy [Corresponds to 'Hard' in PNAS],537.0,294.0,1.826531,eiZmgSepzvcWzXZnq,AB HAVE TO BE NEIGHBORS,2021-03-11 18:33:26.463000+00:00,2.0
4617,zwtfREfmLE7LzyW7n_easy,1,High,Mixed,Mixed,Mixed,High,4,Easy [Corresponds to 'Hard' in PNAS],537.0,294.0,1.826531,ReHqydvhHuWfF9wbc,can we please put it back,2021-03-11 18:33:29.370000+00:00,2.0


In [203]:
create_chunks(juries, 3)

,conversation_num,batch_num,round_num,speaker_nickname,timestamp,message,majority_pct,num_flipped,flipped_pct,num_votes,chunk
0,0,0,0,5e7e1e0031f4e454e196c30b,2020-04-20 18:27:20.125000+00:00,Hello!,1.000000,1,0.333333,3,0.0
1,0,0,0,5e31d6e4e31c5304c46f1413,2020-04-20 18:27:23.764000+00:00,Hi!,1.000000,1,0.333333,3,0.0
2,0,0,0,5e7e4f4c31f4e454e196c9c4,2020-04-20 18:27:27.724000+00:00,Hello,1.000000,1,0.333333,3,0.0
3,0,0,0,5d482ea421c9be351f762255,2020-04-20 18:27:30.410000+00:00,Hi,1.000000,1,0.333333,3,0.0
4,0,0,0,5e84cc3c50f6e364321d6265,2020-04-20 18:27:35.506000+00:00,hi,1.000000,1,0.333333,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...
14883,347,177,3,5f1f466846a7f03dda9b37c3,2020-07-28 22:36:22.226000+00:00,I think he is the asshole,0.666667,1,0.333333,3,2.0
14884,347,177,3,5f1a214246a7f03dda9a6aad,2020-07-28 22:36:46.155000+00:00,He didn't vent to them or take up frustrations...,0.666667,1,0.333333,3,2.0
14885,347,177,3,5f2096dab0aafc741f6865bb,2020-07-28 22:36:49.238000+00:00,very interested,0.666667,1,0.333333,3,2.0
14886,347,177,3,5f1f472646a7f03dda9b3895,2020-07-28 22:36:51.487000+00:00,"I'm fine with venting on Reddit, I just don't ...",0.666667,1,0.333333,3,2.0
